In [59]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2016,23)

In [68]:
input_text="""cpy 2 a
tgl a
tgl a
tgl a
cpy 1 a
dec a
dec a
"""
input_lines = input_text.split('\n')

In [86]:
import time
import re

registers = {
    'a': 0,
    'b': 0,
    'c': 0,
    'd': 0
}

instructions = []
instructions.append('inc')
instructions.append('dec')
instructions.append('cpy')
instructions.append('jnz')
instructions.append('tgl')

pattern_instruction = re.compile(r'(\w+) (\S+) *(\S+)?')
pattern_jump_value = re.compile(r'([+-])?(\d+)')

def decode_program(input_text):
    """
    Function to parse text block the program instructions

    parameters:
        uinput_text: string name of data

    returns:
        program: list of dict, program instructions
    """
    program = []
    # split text into lines
    lines = input_text.split('\n')

    # process each line
    for line in lines:
        # instruction regex r'(\w+) (\S+) *(\S+)?'
        matches = pattern_instruction.match(line)
        if matches:
            instruction = matches.group(1)
            # simple instructions inc, and dec,
            #   just store instruction and register
            if instruction in ['inc','dec']:
                register = matches.group(2)
                program.append({
                    'instruction': instruction,
                    'x': register
                })
            elif instruction in ['tgl']:
                register = matches.group(2)
                program.append({
                    'instruction': instruction,
                    'x': register
                })
            # copy is more comples, source can be an int or register
            elif instruction in ['cpy']:
                source = matches.group(2)
                # is it a number, if so convert to int?
                if source.isdigit():
                    source = int(source)
                # target should be a register
                register = matches.group(3)
                program.append({
                    'instruction': instruction,
                    'x': source,
                    'y': register
                })
            # jnz, also uses int or register
            elif instruction in ['jnz']:
                val_x = matches.group(2)
                # int? convert it
                if val_x.isdigit():
                    val_x = int(val_x)
                # split val_y with jump regex r'([+-])?(\d+)'
                val_y = matches.group(3)
                matches2 = pattern_jump_value.match(val_y)
                direction = matches2.group(1) or '+'
                val_y = int(matches2.group(2))
                # if direction is - convert val_y to negative
                if direction == '-':
                    val_y*=-1
                program.append({
                    'instruction': instruction,
                    'x': val_x,
                    'y': val_y
                })
    return program

def run_program(program):
    """
    Executes the program instructions in PROGRAM
    """
    print("Running")
    # pointer for current program location
    pointer = 0
    # while pointer is valid, keep processing
    while 0 <= pointer < len(program):
        current_line = program[pointer]
        print(pointer, current_line)
        # increase by one, and move pointer +1
        if current_line['instruction'] == 'inc':
            registers[current_line['x']]+=1
            pointer+=1
        # decrease by 1 and move pointe +1
        elif current_line['instruction'] == 'dec':
            registers[current_line['x']]-=1
            pointer+=1
        # copy source to target, and move pointer +1
        elif current_line['instruction'] == 'cpy':
            try:
                # is int?
                if isinstance(current_line['x'],int):
                    # yes, use source value
                    registers[current_line['y']] = current_line['x']
                elif current_line['x'] in registers:
                    # no, get register value
                    registers[current_line['y']] = registers[current_line['x']]
            except Exception as error_message:
                print(error_message)
            pointer+=1
        # jnz jumps value instructions if value is not 0
        elif current_line['instruction'] == 'jnz':
            x_value = current_line['x']
            # is int?
            if not isinstance(current_line['x'],int):
                # no, get value from register
                x_value = registers[current_line['x']]
            # is zero?
            if x_value != 0:
                # No, jump
                if current_line['y'] in registers:
                    pointer += registers[current_line['y']]
                else:
                    pointer += current_line['y']
            else:
                # Yes, move forward 1
                pointer+=1
        # tgl x toggles the instruction x away (pointing at instructions like
        # jnz does: positive means forward; negative means backward):
        elif current_line['instruction'] == 'tgl':
            if current_line['x'] in registers:
                target_pointer = pointer + registers[current_line['x']]
            else:
                target_pointer = pointer + int(current_line['x'])
            print(f"tgl {target_pointer}: {program[target_pointer]}")
            pointer+=1
            try:
                target = program[target_pointer]
                # For one-argument instructions, inc becomes dec, and all other one-argument
                # instructions become inc.
                if target['instruction'] == 'dec':
                    target['instruction'] = 'inc'
                elif target['instruction'] == 'inc':
                    target['instruction'] = 'dec'
                # If tgl toggles itself (for example, if a is 0, tgl a would target itself and
                # become inc a), the resulting instruction is not executed until the next time
                # it is reached.
                elif target['instruction'] == 'tgl':
                    target['instruction'] = 'inc'
                #For two-argument instructions, jnz becomes cpy, and all other two-instructions become jnz.
                elif target['instruction'] == 'jnz':
                    target['instruction'] = 'cpy'
                elif target['instruction'] == 'cpy':
                    target['instruction'] = 'jnz'
                #If an attempt is made to toggle an instruction outside the program, nothing happens.
                #If toggling produces an invalid instruction (like cpy 1 2) and an attempt is later made to execute that instruction, skip it instead.
            except Exception as error_message:
                print(error_message)


In [88]:
program

[{'instruction': 'cpy', 'x': 2, 'y': 'a'},
 {'instruction': 'tgl', 'x': 'a'},
 {'instruction': 'tgl', 'x': 'a'},
 {'instruction': 'inc', 'x': 'a'},
 {'instruction': 'jnz', 'x': 1, 'y': 'a'},
 {'instruction': 'dec', 'x': 'a'},
 {'instruction': 'dec', 'x': 'a'}]

In [92]:
registers = {
    'a': 7,
    'b': 0,
    'c': 0,
    'd': 0
}
program = decode_program(input_text)
print(program)
run_program(program)
registers

[{'instruction': 'cpy', 'x': 2, 'y': 'a'}, {'instruction': 'tgl', 'x': 'a'}, {'instruction': 'tgl', 'x': 'a'}, {'instruction': 'tgl', 'x': 'a'}, {'instruction': 'cpy', 'x': 1, 'y': 'a'}, {'instruction': 'dec', 'x': 'a'}, {'instruction': 'dec', 'x': 'a'}]
Running
0 {'instruction': 'cpy', 'x': 2, 'y': 'a'}
1 {'instruction': 'tgl', 'x': 'a'}
tgl 3: {'instruction': 'tgl', 'x': 'a'}
2 {'instruction': 'tgl', 'x': 'a'}
tgl 4: {'instruction': 'cpy', 'x': 1, 'y': 'a'}
3 {'instruction': 'inc', 'x': 'a'}
4 {'instruction': 'jnz', 'x': 1, 'y': 'a'}


{'a': 3, 'b': 0, 'c': 0, 'd': 0}